In [1]:
# Imports
import gymnasium as gym
from typing import Callable

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env import VecMonitor
import torch as th

import pickle

# Import Our environment
from dev_env import tradingEng


c:\Users\osc16\miniconda3\envs\d2d\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# Load Paths
with open("1.6kRunDemo.pkl","rb") as fp:
    paths = pickle.load(fp)

paths1 = paths[0:300]
paths2 = paths[300:600]
paths3 = paths[600:900]
paths4 = paths[900:1200]
del(paths)

In [3]:
## LR schedule
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [4]:
# Def Env
def start_and_release(set, action = 'small-More-Trust', obs = 'big'):
    ret = tradingEng(set, action = action, obs = obs)
    del(set)
    return ret

envs = VecMonitor(SubprocVecEnv([
    lambda: start_and_release(paths1,action = 'small-More-Trust',obs = 'auto'),
    lambda: start_and_release(paths2,action = 'small-More-Trust',obs = 'auto'),
    lambda: start_and_release(paths3,action = 'small-More-Trust',obs = 'auto'),
    lambda: start_and_release(paths4,action = 'small-More-Trust',obs = 'auto'), #4
], start_method='spawn'))

print("Got here?")

# Instantiate the agent
policy_kwargs = dict(activation_fn=th.nn.LeakyReLU,
                     net_arch=dict(pi=[512,512,256,128,64,64,64,64,36,18], vf=[512,512,256,128,64,64,64,64,36,18], optimizers_class = th.optim.Adam, log_std_init = False, ortho_init = False))
model = PPO("MlpPolicy", envs, batch_size = 1200, learning_rate=linear_schedule(3e-4), policy_kwargs=policy_kwargs, n_steps=1200*5, normalize_advantage=True, vf_coef = 0.9, gamma = 0.9,ent_coef=0.01, verbose = 1)

#print("Number 2")
#model = PPO.load("PPOs")
#model.set_env(envs)
#model.learning_rate = linear_schedule(5e-5)
#model.batch_size = 3000
#model.ent_coef = 0.005
#model.learn(total_timesteps=int(2e6), log_interval=1)

#print("Last Step")
model.learn(total_timesteps=int(5e6), log_interval=1)
# Save the agent
model.save("PPOs-Auto")

Got here?
Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 652      |
|    ep_rew_mean     | -977     |
| time/              |          |
|    fps             | 1480     |
|    iterations      | 1        |
|    time_elapsed    | 16       |
|    total_timesteps | 24000    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 652          |
|    ep_rew_mean          | -899         |
| time/                   |              |
|    fps                  | 1178         |
|    iterations           | 2            |
|    time_elapsed         | 40           |
|    total_timesteps      | 48000        |
| train/                  |              |
|    approx_kl            | 0.0024021713 |
|    clip_fraction        | 0.00827      |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.81        |
|    explained_variance   | 0.0

In [14]:
import numpy as np
print(env._get_obs())
path = paths[800]
lambdax = path.lambdas[25]
r = path.r[25]
t = path.t_s[25]
print(lambdax)
print(r)

#model.predict(observation=np.asmatrix([lambdax,r,t]),state=None, deterministic=True)

[[-0.99977158  0.05536838 -0.56841282]]
0.005196713067477058
0.030745992173113156
